# **Libraries**

In [6]:
# Basic packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import seaborn as sns
import os, pickle
from zipfile import ZipFile
from warnings import filterwarnings

# Machine learning packages
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from tensorflow import keras

# Performance metrics
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error, r2_score
from sklearn.model_selection import train_test_split, TimeSeriesSplit

filterwarnings('ignore')

In [25]:
window = 50                # Size of array
predictionHorizonMax = 12  # Maximum prediction horizon
neurons = 64               # Number of neurons in LSTM


#  **Functions**

In [5]:
def SerieMatriz(timeSerie, predictionHorizonMax, window):
  timeSerie = np.squeeze(timeSerie)

  X = np.zeros([len(timeSerie)-predictionHorizonMax-window+1, window])
  y = np.zeros([len(timeSerie)-predictionHorizonMax-window+1, predictionHorizonMax])
  for i in range(X.shape[0]):
      X[i,:] = timeSerie[i:i+window]
      y[i,:] = timeSerie[i+window: i+window+predictionHorizonMax]
  
  return X, np.squeeze(y)

# **Read datasets**

In [ ]:
path = '/Users/guane/Documentos/Doctorate/GuajiraSustainableWindBot/data/raw'
datasets = os.listdir(path)

data = {}
for csv_file in datasets:

  name = csv_file.split('.')[0]
  print("======"*8, csv_file)
  
  # Read dataset
  df = pd.read_csv(path + '/' + csv_file)

  timeSerie = df['wind_speed_10m']
  time = np.arange(0, len(timeSerie),1)

  # Create time series matrix
  #X, y = SerieMatriz(timeSerie, predictionHorizonMax, window)

  #X_, X_test, y_, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  #X_train, X_valid, y_train, y_valid = train_test_split(X_, y_, test_size=0.2, random_state=42)

  #X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
  #X_valid = np.reshape(X_valid, (X_valid.shape[0], X_valid.shape[1], 1))
  #X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))







  


================================================ open_meteo_la_jagua_del_pilar.csv
================================================ open_meteo_san_juan_del_cesar.csv
================================================ open_meteo_hatonuevo.csv
================================================ open_meteo_riohacha.csv
================================================ open_meteo_albania.csv
================================================ open_meteo_uribia.csv
================================================ open_meteo_manaure.csv
================================================ open_meteo_distraccion.csv
================================================ open_meteo_el_molino.csv
================================================ open_meteo_mingueo.csv
================================================ open_meteo_maicao.csv
================================================ open_meteo_barrancas.csv
================================================ open_meteo_fonseca.csv


In [ ]:
}

In [ ]:
for 